In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

import time
from os import listdir
from os.path import isfile, join
import shutil

import argparse
from datetime import datetime, timedelta
import pandas as pd

In [3]:
path=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\AutomatizacionExito\chromedriver_win32\chromedriver.exe"
mypathD=r"C:\Users\jcleiva\Downloads"
mypathPDL=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\P&G Industria\V3\Ingresos\Ingresos"
service = Service(executable_path=path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://pasarela.grupo-exito.com/MicroStrategy/servlet/mstrWeb?evt=3001&src=mstrWeb.3001&Port=0&")

usr=driver.find_element(By.XPATH,'//*[@id="Uid"]')
usr.send_keys("jcleiva")
psw=driver.find_element(By.XPATH,'//*[@id="Pwd"]')
psw.send_keys("Teb.1030611534")

driver.find_element(By.XPATH,'//*[@id="3054"]').click()
driver.find_element(By.XPATH,'//*[@id="projects_ProjectsStyle"]/table/tbody/tr/td[1]/div/table/tbody/tr/td[2]/a').click()
driver.find_element(By.XPATH,'//*[@id="dktpSectionView"]/a[2]/div[1]').click()
driver.find_element(By.XPATH,'//*[@id="FolderIcons"]/tbody/tr[1]/td[1]/div/table/tbody/tr/td[2]/a').click()

tiempo=(2023,8)
m

In [8]:
m=8
y=2023

In [6]:
time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="id_mstr116"]/table/tbody/tr[8]/td[2]').click()
time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="id_mstr329"]/div[2]/div/div/div[2]/div/a[4]').click()
driver.find_element(By.XPATH,'//*[@id="id_mstr460_txt"]').clear()

In [9]:
meses={1:"Enero",2:"Febrero",3:"Marzo",4:"Abril",5:"Mayo",6:"Junio",7:"Julio",8:"Agosto",9:"Septiembre",10:"Octubre",
      11:"Noviembre",12:"Diciembre"}
driver.find_element(By.XPATH,'//*[@id="id_mstr460_txt"]').send_keys("{} {}".format(meses[m],y))

In [11]:
driver.find_element(By.XPATH,'//*[@id="id_mstr462"]').click()

In [12]:
driver.find_element(By.XPATH,'//*[@id="id_mstr284"]').click()

original_window=driver.current_window_handle

time.sleep(50)

driver.find_element(By.XPATH,'//*[@id="tbExport"]').click()
time.sleep(3)

driver.switch_to.window(driver.window_handles[-1])
prefiles = [f for f in listdir(mypathD) if isfile(join(mypathD, f)) and f[-5:]==".xlsx"]

driver.find_element(By.XPATH,'//*[@id="3131"]').click()

time.sleep(30)

posfiles = [f for f in listdir(mypathD) if isfile(join(mypathD, f)) and f[-5:]==".xlsx"]

nFiles=list(set(posfiles) - set(prefiles))
if len(nFiles)==1:
    shutil.copy(join(mypathD, nFiles[0]),join(mypathPDL, "{}{:02d}. Ingresos.xlsx".format(y,m)))
else:
    print("No pudimos determinar el archivo nuevo: {} {:02d}".format(y,m))

driver.close()

driver.switch_to.window(original_window)
driver.find_element(By.XPATH,'//*[@id="tbBack0"]').click()

time.sleep(3)
print("Reporte de Ingresos descargado: {} {}".format(y,m))
titlesReport(mypathPDL, "{}{:02d}. Ingresos.xlsx".format(y,m),"Ingresos")

Reporte de Ingresos descargado: 2023 8


NameError: name 'titlesReport' is not defined

In [4]:
import openpyxl

In [5]:
fname=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\P&G Industria\V3\Despachos\202108. Despachos - copia2.xlsx"

In [6]:
sname="Despachos y Devoluciones Indust"

In [7]:
book = openpyxl.load_workbook(fname)

# select the sheet
sheet = book[sname]

In [1]:
import pandas as pd
from os.path import join

In [2]:
columns=['Dependencia Despacha', 'Desc. Dependencia Despacha', 'Orden Despacho/Devolucion',
         'Cod Instalacion', 'Desc. Cod Instalacion', 'Dependencia Recibe', 'Desc. Dependencia Recibe',
         'Sublinea', 'Desc. Sublinea', 'Plu', 'Desc. Plu', 'Flujo Logístico',       
         'Planta Industria', 'Desc. Planta Industria', '# Unidades Despachadas',
         '$ CtoCantDespachada']
conv={"Dependencia Despacha":str,"Orden Despacho/Devolucion":str,"Cod Instalacion":str,"Dependencia Recibe":str,"Sublinea":str,"Plu":str}
rows=1

In [3]:
path=r"C:\Users\jcleiva\Documents\Reportes Base\2023\P&G\Despachos"
filename="Despachos y Devoluciones Industria (2).xlsx"

In [4]:
import win32com.client
import os

In [6]:
xl = win32com.client.DispatchEx("Excel.Application")
wb = xl.workbooks.open(join(path,filename))
xl.Visible = True
wb.sheets("Despachos y Devoluciones Indust").cells(1,1).Value="Test"
#wb.Save
wb.Close(SaveChanges=1)
xl.Quit()

In [7]:
pd.read_excel(join(path,filename),skiprows=48756,header=None,names=columns,converters=dict(zip(columns,[str]*len(columns))),nrows=1)

,Dependencia Despacha,Desc. Dependencia Despacha,Orden Despacho/Devolucion,Cod Instalacion,Desc. Cod Instalacion,Dependencia Recibe,Desc. Dependencia Recibe,Sublinea,Desc. Sublinea,Plu,Desc. Plu,Flujo Logístico,Planta Industria,Desc. Planta Industria,# Unidades Despachadas,$ CtoCantDespachada
0,910,CEDI VIRTUAL DESPACHOS MEDELLI,2021641851,39,NA ...,33,EXITO POBLADO,88,PANADERIA,738851,PAN PRIMAVERA QUESO PREFORMADO,ALMACENAMIENTO,7080,PLANTA PANADERIA MEDELLIN,120,232080
